DATA ANALYTICS

In [1]:
import folium
import pandas as pd
import numpy as np

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
hotel_info = pd.read_csv("../data/input/dataset/hotel_info.csv")
hotel_info.describe()
# hotel_info[hotel_info["hotel_id"] == 202777]
hotel_info.dtypes

hotel_id        int64
hotel_name     object
chain_id        int64
latitude      float64
longitude     float64
dtype: object

In [3]:
chain_info = pd.read_csv("../data/input/dataset/chain_info.csv")
chain_info.dtypes

chain_id       int64
chain_name    object
dtype: object

In [35]:
train_set = pd.read_csv('../data/input/dataset/train_set.csv', header=None, 
                       names=['image_id', 'hotel_id', 'url', 'source', 'timestamp'])
train_set.describe()

# display(train_set)

# train_set["image_id"] = train_set["image_id"].astype(int)
# train_set["hotel_id"] = train_set["hotel_id"].astype(int)

# train_set.dtypes

/tmp/ipykernel_408/619540170.py:1: DtypeWarning:

Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.



,image_id,hotel_id,url,source,timestamp
count,1124216,1124216,1124216,1124216,1124216
unique,1055786,60322,1055786,3,26459
top,9484588,25050,https://i.travelapi.com/hotels/13000000/129900...,travel_website,2019-12-24 21:10:28
freq,4563,474,4563,1079585,68445


In [5]:
data_df = train_set.merge(hotel_info, on="hotel_id")
data_df = data_df.merge(chain_info, on="chain_id")
# data_df["image_id"] = data_df["image_id"].astype(str)
# data_df["hotel_id"] = data_df["hotel_id"].astype(str)
# data_df["chain_id"] = data_df["chain_id"].astype(str)

display(data_df.head())
len(data_df)

,image_id,hotel_id,url,source,timestamp,hotel_name,chain_id,latitude,longitude,chain_name
0,3485,18187,https://traffickcam.com/images/2016/10/2015090...,traffickcam,9/9/15 17:23,Hilton Minneapolis,3,44.97338,-93.27331,Hilton
1,3486,18187,https://traffickcam.com/images/2016/10/2015090...,traffickcam,9/9/15 17:23,Hilton Minneapolis,3,44.97338,-93.27331,Hilton
2,2599229,18187,https://traffickcam.com/images/2017/7/20160627...,traffickcam,6/27/16 21:27,Hilton Minneapolis,3,44.97338,-93.27331,Hilton
3,2599231,18187,https://traffickcam.com/images/2017/7/20160627...,traffickcam,6/27/16 21:27,Hilton Minneapolis,3,44.97338,-93.27331,Hilton
4,2599232,18187,https://traffickcam.com/images/2017/7/20160627...,traffickcam,6/27/16 21:27,Hilton Minneapolis,3,44.97338,-93.27331,Hilton


599999

In [6]:
# merged = train_set.merge(hotel_info, on='hotel_id', how='outer')
# merged = merged.merge(chain_info, on='chain_id', how='outer')

# missing_rows = merged[merged.isnull().any(axis=1)]
# missing_rows

In [7]:
import os
import glob

# Define the path to the folder containing the downloaded images
image_train_folder_path = "../data/images/train/*"
image_test_folder_path = "../data/images/test/*"


# Get a list of all the filenames in the image folder
image_train_filenames = glob.glob(image_train_folder_path)
image_test_filenames = glob.glob(image_test_folder_path)
image_filenames = image_train_filenames + image_test_filenames


# Convert the filenames to a set for faster lookups
image_ids = set([int(filename.split(".")[-2].split("/")[-1]) for filename in image_filenames])

# Create a boolean array indicating which images are available
image_available = np.array([x in image_ids for x in data_df["image_id"]], dtype=np.int8)

# Add the image_available column to the data_df dataframe
data_df["image_available"] = image_available
len(data_df[data_df["image_available"] == 1])
print(f"{len(image_filenames)}, {len(image_ids)}")

data_df.head()

509873, 507572


,image_id,hotel_id,url,source,timestamp,hotel_name,chain_id,latitude,longitude,chain_name,image_available
0,3485,18187,https://traffickcam.com/images/2016/10/2015090...,traffickcam,9/9/15 17:23,Hilton Minneapolis,3,44.97338,-93.27331,Hilton,0
1,3486,18187,https://traffickcam.com/images/2016/10/2015090...,traffickcam,9/9/15 17:23,Hilton Minneapolis,3,44.97338,-93.27331,Hilton,1
2,2599229,18187,https://traffickcam.com/images/2017/7/20160627...,traffickcam,6/27/16 21:27,Hilton Minneapolis,3,44.97338,-93.27331,Hilton,1
3,2599231,18187,https://traffickcam.com/images/2017/7/20160627...,traffickcam,6/27/16 21:27,Hilton Minneapolis,3,44.97338,-93.27331,Hilton,1
4,2599232,18187,https://traffickcam.com/images/2017/7/20160627...,traffickcam,6/27/16 21:27,Hilton Minneapolis,3,44.97338,-93.27331,Hilton,1


In [8]:
image_ids_train = set([filename.split(".")[-2].split("/")[-1] for filename in image_train_filenames])
image_ids_test = set([filename.split(".")[-2].split("/")[-1] for filename in image_test_filenames])

# ((len(image_test_filenames)-2000)/len(image_train_filenames))*100
len(image_train_filenames) - len(list(set(image_ids_train).intersection(image_ids_test)))

# len(data_df["image_id"]) - len(data_df["image_id"].drop_duplicates())

406060

In [9]:
chain_group_df = data_df.groupby(["chain_name"]).agg({"hotel_id": [pd.Series.nunique], "image_id": [pd.Series.nunique], "image_available": ["sum"]})
chain_group_df.columns = ["_".join(x) for x in chain_group_df.columns.ravel()]
chain_group_df = chain_group_df.reset_index().sort_values("hotel_id_nunique", ascending=False)

# chain_group_df["image_available_sum"].sum() / chain_group_df["image_id_nunique"].sum()
chain_group_df.dtypes
chain_group_df = chain_group_df[0:20]

chain_group_df["image_available_sum_fr"] = (chain_group_df["image_available_sum"] / chain_group_df["image_id_nunique"]) * (chain_group_df["image_id_nunique"] / chain_group_df["image_id_nunique"].max()) * 75
display(chain_group_df)
chain_group_df["image_available_sum"].max()

/tmp/ipykernel_408/3171732171.py:2: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  chain_group_df.columns = ["_".join(x) for x in chain_group_df.columns.ravel()]


,chain_name,hotel_id_nunique,image_id_nunique,image_available_sum,image_available_sum_fr
90,unknown,13952,329666,289027,65.754506
35,Holiday Inn,1777,28924,26128,5.944198
32,Hampton,1211,18494,8135,1.850737
14,Comfort Inn,715,12581,8535,1.941738
7,Best Western,655,12249,8254,1.877810
20,Days Inn,622,7059,4950,1.126140
65,Quality Inn,585,9827,7199,1.637794
82,Super 8,527,5448,4276,0.972803
51,Marriott,475,9049,8605,1.957663
17,Courtyard by Marriott,444,5076,4652,1.058344


289027

In [34]:
fig = px.scatter(chain_group_df, x="chain_name", y="hotel_id_nunique",
                 size="image_id_nunique", color = "image_id_nunique",
                 hover_name = None,
                 log_y=False, size_max=75)

fig.update_yaxes(title_text="Hotel count", tickfont_size=18, title_font_size=20)
fig.update_xaxes(title_text="Chain ID", tickfont_size=18, title_font_size=20)
fig.update_layout(coloraxis=dict(colorbar=dict(title="Image count")))
fig.update_traces(hovertemplate="Chain: %{x} <br>Hotel count: %{y:%d}<br>Image count: %{marker.size:%d}")
fig.show()
fig.write_image("./hotes_and_image_count_per_chain.png", width=1600, height=400)

In [11]:
# # Add a second circle for the available images
# fig = px.scatter(chain_group_df, x="chain_name", y="hotel_id_nunique",
#                  size="image_available_sum",
#                  color="image_id_nunique",
#                  opacity=1,
#                  hover_name=None,
#                  log_y=False, max_size=75)

# # fig.add_scatter(x=chain_group_df["chain_name"], y=chain_group_df["hotel_id_nunique"], mode="markers",
# #                 marker=dict(size=chain_group_df["image_available_sum"]/2000, color=chain_group_df["image_id_nunique"], opacity=1),
# #                  showlegend=False)
# # fig.add_scatter(fig.add_scatter(x=chain_group_df["chain_name"], y=chain_group_df["hotel_id_nunique"],
# #                  mode="markers", marker=dict(size=chain_group_df["image_id_nunique"], color=chain_group_df["image_id_nunique"], opacity=0.1),
# #                  showlegend=False))



# # fig.update_yaxes(title_text="Hotel count", type="log")
# fig.update_xaxes(title_text="Chain ID", tickmode='linear', tickangle=90)
# fig.update_layout(title="Hotel and image count per chain", coloraxis=dict(colorbar=dict(title="Image count")))

# # Update the hovertemplate to include the available image count
# fig.update_traces(hovertemplate="Chain: %{x} <br>Hotel count: %{y:%d}<br>Total image count: %{marker.size:%d}" +
#                   "<br>Available image count: %{marker.color}/%{marker.size}")
# fig.show()

TypeError: scatter() got an unexpected keyword argument 'max_size'

In [14]:
group_df = data_df.groupby(["hotel_id"]).size().to_frame("image_count").sort_values("image_count")[::-1].reset_index()

In [22]:
fig = px.histogram(group_df, x="image_count", nbins=100, marginal="box", height=500)
# fig.update_layout(title="Distribution of image count per hotel")
# fig.update_traces(hovertemplate="Image count: %{x} <br>Hotel count: %{y:%d}")
fig.update_yaxes(title_text="Hotel count")
fig.update_xaxes(title_text="Image count")
fig.show()
fig.write_image("./distribution_of_image_count_per_hotel.jpg")
